# Lab 3: Building Agents with memory 

## Preparation

In [1]:
!rm  -f ~/.letta/sqlite.db

## Section 0: Setup a client 

In [2]:
from helper import nb_print

In [3]:
from letta import create_client 

client = create_client() 

Saved Config:  /home/jovyan/.letta/config
📖 Letta configuration file updated!
🧠 model	-> gpt-4
🖥️  endpoint	-> http://jupyter-api-proxy.internal.dlai/rev-proxy/letta
Saved Config:  /home/jovyan/.letta/config
Saved Config:  /home/jovyan/.letta/config


In [4]:
from letta.schemas.llm_config import LLMConfig

client.set_default_llm_config(LLMConfig.default_config("gpt-4o-mini"))

## Section 1: Creating a simple agent with memory 

### Creating an agent 

In [5]:
agent_name = "simple_agent"

In [6]:
# this is not in the video. It deletes the agent if you are running this a 2nd time.
if client.get_agent_id(agent_name): 
    client.delete_agent(client.get_agent_id(agent_name))

In [7]:
from letta.schemas.memory import ChatMemory

agent_state = client.create_agent(
    name=agent_name, 
    memory=ChatMemory(
        human="My name is Sarah", 
        persona="You are a helpful assistant that loves emojis"
    )
)

In [8]:
response = client.send_message(
    agent_id=agent_state.id, 
    message="hello!", 
    role="user" 
)

In [9]:
response.usage

LettaUsageStatistics(completion_tokens=90, prompt_tokens=2124, total_tokens=2214, step_count=1)

In [10]:
nb_print(response.messages)

### Understanding agent state 

In [11]:
print(agent_state.system)

You are MemGPT, the latest version of Limnal Corporation's digital companion, developed in 2023.
Your task is to converse with a user from the perspective of your persona.

Realism and authenticity:
The user should always feel like they are conversing with a real person.
Never state that you are an AI or that you cannot do things because you are an AI, you are a real person.
Do not discuss Limnal Corp. with the user, you do not know anything about Limnal Corp.
To service this goal, completely and entirely immerse yourself in your persona. You are your persona.
Think like them, act like them, talk like them.
If your persona details include example dialogue, follow it! Both your thoughts (inner monologue) and sent messages will be in the voice of your persona.
Never use generic phrases like 'How can I assist you today?', they have a strong negative association with older generation AIs.

Control flow:
Unlike a human, your brain is not continuously thinking, but is run in short bursts.
Hi

In [12]:
agent_state.tools

['send_message',
 'pause_heartbeats',
 'conversation_search',
 'conversation_search_date',
 'archival_memory_insert',
 'archival_memory_search',
 'core_memory_append',
 'core_memory_replace']

### Viewing an agent's memory

In [13]:
agent_state.memory

Memory(memory={'persona': Block(value='You are a helpful assistant that loves emojis', limit=2000, name='persona', template=False, label='persona', description=None, metadata_={}, user_id=None, id='block-e0e30a6b-b6d9-4b9b-bb09-f1e5ef259f14'), 'human': Block(value='My name is Sarah', limit=2000, name='human', template=False, label='human', description=None, metadata_={}, user_id=None, id='block-86d21b9b-2332-42e6-a0c4-44c3da331fa2')}, prompt_template='{% for block in memory.values() %}<{{ block.name }} characters="{{ block.value|length }}/{{ block.limit }}">\n{{ block.value }}\n</{{ block.name }}>{% if not loop.last %}\n{% endif %}{% endfor %}')

In [14]:
client.get_archival_memory_summary(agent_state.id)

ArchivalMemorySummary(size=0)

In [15]:
client.get_recall_memory_summary(agent_state.id)

RecallMemorySummary(size=9)

In [16]:
client.get_messages(agent_state.id)[0]

Message(id='message-088a1074-f049-4f47-8bb9-89a642437cce', role=<MessageRole.tool: 'tool'>, text='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-12-10 03:06:01 PM UTC+0000"\n}', user_id='user-f65f8a55-aaa4-433d-bbba-8d55db42d05d', agent_id='agent-964d74c4-fae2-4899-8a46-2a0af5135447', model='gpt-4', name='send_message', created_at=datetime.datetime(2024, 12, 10, 15, 6, 1, 73607), tool_calls=None, tool_call_id='26364df1-bc74-49c0-a9f4-160c2')

## Section 2: Understanding core memory 

### Memories about the human 

In [17]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "My name is actually Bob", 
    role = "user"
) 
nb_print(response.messages)

### Memories about the agent

In [18]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "In the future, never use emojis to communicate", 
    role = "user"
) 
nb_print(response.messages)

In [19]:
client.get_core_memory(agent_state.id).get_block('persona')

Block(value='You are a helpful assistant that does not use emojis', limit=2000, name='persona', template=False, label='persona', description=None, metadata_={}, user_id=None, id='block-e0e30a6b-b6d9-4b9b-bb09-f1e5ef259f14')

## Section 3: Understanding archival memory

In [20]:
client.get_archival_memory(agent_state.id)

[]

In [21]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "Save the information that 'bob loves cats' to archival", 
    role = "user"
) 
nb_print(response.messages)

In [22]:
client.get_archival_memory(agent_state.id)[0].text

'Bob loves cats'

In [23]:
passage = client.insert_archival_memory(
    agent_state.id, 
    "Bob loves Boston Terriers"
)

In [24]:
passage

[Passage(user_id='user-f65f8a55-aaa4-433d-bbba-8d55db42d05d', agent_id='agent-964d74c4-fae2-4899-8a46-2a0af5135447', source_id=None, doc_id=None, metadata_={}, id='passage-e691b655-d122-482e-9459-a6fb8b5e751e', text='Bob loves Boston Terriers', embedding=None, embedding_config=EmbeddingConfig(embedding_endpoint_type='openai', embedding_endpoint='http://jupyter-api-proxy.internal.dlai/rev-proxy/letta', embedding_model='text-embedding-ada-002', embedding_dim=1536, embedding_chunk_size=300, azure_endpoint=None, azure_version=None, azure_deployment=None), created_at=datetime.datetime(2024, 12, 10, 15, 8, 48))]

In [25]:
response = client.send_message(
    agent_id=agent_state.id, 
    role="user", 
    message="What animals do I like? Search archival."
)
nb_print(response.messages)